<a href="https://colab.research.google.com/github/Moneshai2004/-genai-Summarize-the-dialogue-using-a-Generative-AI-model-like-FLAN-T5/blob/main/Summarize_the_dialogue_using_a_Generative_AI_model_like_FLAN_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch

In [ ]:
from datasets import load_dataset

dataset = load_dataset("knkarthick/dialogsum")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Step 4: Tokenize Input Dialogue & Generate Summary (Zero-Shot)

In [ ]:
# Sample dialogue
dialogue = """
Person1: What time is it, Tom?
Person2: Just a minute, it's 10 to 9 by my watch.
Person1: Are you sure? We need to catch the train.
Person2: Don’t worry, we have plenty of time.
"""

# Add instruction for the model
prompt = "Summarize the following conversation:\n" + dialogue + "\nSummary:"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate summary
outputs = model.generate(**inputs, max_length=60)

# Decode the output tokens to text
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:")
print(summary)


Generated Summary:
Tom and Tom are going to catch the train.


In [ ]:
# Prepare input with prompt
dialogue = "Person 1: What time is it, Tom?\nPerson 2: It's 9:50.\n"

input_text = "Summarize the following conversation:\n" + dialogue + "\nSummary:"

inputs = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(
    inputs["input_ids"],
    max_length=50,
    num_beams=2,      # use beam search for better quality
    early_stopping=True
)

summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Summary:")
print(summary)


Generated Summary:
The time is 9:50.


# One-shot example dialogue + summary (the shot)

In [ ]:

# One-shot example dialogue + summary (the shot)
example_dialogue = """
Person 1: What time is it, Tom?
Person 2: It's 9:50 by my watch.
"""
example_summary = "Person 2 tells Person 1 that the time is 9:50."

# New dialogue to summarize
new_dialogue = """
Person 1: Is the train leaving soon?
Person 2: Yes, it's about to leave in 5 minutes.
"""

# Create prompt with one-shot example and new dialogue
prompt = f"Summarize the following conversations.\n\nDialogue:\n{example_dialogue}\nSummary:\n{example_summary}\n\nDialogue:\n{new_dialogue}\nSummary:"

# Tokenize prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate summary
outputs = model.generate(**inputs, max_length=50, num_beams=4, early_stopping=True)

# Decode and print summary
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Summary:", summary)



Generated Summary: The train is about to leave in 5 minutes.


 few-shot prompting example in Python with FLAN-T5:
python

In [ ]:

few_shot_prompt = """
Dialogue:
Person 1: What time is it?
Person 2: It's 9:50.

Summary:
The time is 9:50.

Dialogue:
Person 1: I need to upgrade my computer.
Person 2: You should add a CD-ROM.

Summary:
Person 1 wants to upgrade their computer by adding a CD-ROM.

Dialogue:
Person 1: Tom, are we going to miss the train?
Person 2: No, we still have 5 minutes.

Summary:
"""

inputs = tokenizer(few_shot_prompt, return_tensors="pt")

outputs = model.generate(**inputs, max_length=60)

summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Summary:")
print(summary)

Generated Summary:
Tom and Tom are going to miss the train.


In [ ]:
from transformers import pipeline, GenerationConfig

# Load summarization pipeline (or model + tokenizer separately)
summarizer = pipeline("summarization", model="google/flan-t5-base")

# Generation config example
gen_config = GenerationConfig(
    max_length=50,   # Limit summary length
    temperature=0.7, # Moderate creativity
    top_p=0.9,       # Nucleus sampling threshold
    top_k=50         # Limits next token candidates
)

# Input dialogue text
few_shot_prompt = """
Dialogue:
Person 1: What time is it?
Person 2: It's 9:50.

Summary:
The time is 9:50.

Dialogue:
Person 1: I need to upgrade my computer.
Person 2: You should add a CD-ROM.

Summary:
Person 1 wants to upgrade their computer by adding a CD-ROM.

Dialogue:
Person 1: Tom, are we going to miss the train?
Person 2: No, we still have 5 minutes.

Summary:
"""

# Generate summary with config
summary = summarizer(few_shot_prompt, generation_config=gen_config)

print(summary[0]['summary_text'])


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Your max_length is set to 200, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Tom and Tom are going to miss the train.


You’ve learned about:

Loading datasets

Using pretrained models like FLAN-T5

Tokenization basics

Zero-shot, one-shot, and few-shot prompting

Prompt engineering basics

Adjusting generation parameters like temperature
